# Features

In [22]:
import numpy as np

from util import get_train_dataset

df = get_train_dataset()
df.head()

,CDR3_alfa,TRAV,TRAJ,CDR3_beta,TRBV,TRBJ,reaction
0,NaN,NaN,NaN,CASSLTSWGNNEQFF,TRBV5-5,TRBJ2-1,0
1,NaN,NaN,NaN,CASSLEATGGTYNEQFF,TRBV5-1,TRBJ2-1,0
2,NaN,NaN,NaN,CASSSPRRGIQETQYF,TRBV5-1*01,TRBJ2-5*01,1
3,CAGPGGSSNTGKLIF,TRAV35*01,TRAJ37*01,CASSLIYPGELFF,TRBV27*01,TRBJ2-2*01,1
4,NaN,NaN,NaN,CASSFEAGATYNEQFF,TRBV7-6,TRBJ2-1,0


In [23]:
feature_functions = []

In [24]:
test_df = df[['CDR3_beta', 'TRBV', 'TRBJ']].rename(columns={'CDR3_beta': 'CDR3', 'TRBV': 'V', 'TRBJ': 'J'})

# One-hot encoding

In [25]:
from sklearn import feature_extraction
import pandas as pd

ONEHOT_ENCODER = feature_extraction.DictVectorizer(sparse=False)

def onehot_encode(df, test=False):
    # One hot encode the columns (creates a new column per unique value here and fills it with 1 or 0)
    onehot_cols = ['V', 'J']
    if not test:
        encodings = ONEHOT_ENCODER.fit_transform(df[onehot_cols].to_dict(orient='records'))
    else:
        encodings = ONEHOT_ENCODER.transform(df[onehot_cols].to_dict(orient='records'))
    onehot_df = pd.DataFrame(encodings, columns=ONEHOT_ENCODER.get_feature_names_out())
    return onehot_df


feature_functions.append(onehot_encode)

onehot_encode(test_df).head()

,J,J=TRBJ1-1,J=TRBJ1-1*01,J=TRBJ1-2,J=TRBJ1-2*01,J=TRBJ1-3,J=TRBJ1-3*01,J=TRBJ1-4,J=TRBJ1-4*01,J=TRBJ1-5,...,V=TRBV7-6*01,V=TRBV7-7,V=TRBV7-7*01,V=TRBV7-8,V=TRBV7-8*01,V=TRBV7-9,V=TRBV7-9*01,V=TRBV7-9*03,V=TRBV9,V=TRBV9*01
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# sequence length

In [26]:
from pyteomics import parser


def get_length(sequence):
    if not isinstance(sequence, str):
        # It's probably NaN
        return 0
    else:
        return parser.length(sequence)


def cdr3_length(df):
    return df['CDR3'].apply(get_length).to_frame('CDR3_length')


feature_functions.append(cdr3_length)

cdr3_length(test_df).head()

,CDR3_length
0,15
1,17
2,16
3,13
4,16


# Number of occurences of each amino acid

In [27]:
def get_amino_acid_composition(sequence):
    if not isinstance(sequence, str):
        # It's probably NaN
        return {}  # {aa: 0 for aa in parser.amino_acids}
    else:
        composition = parser.amino_acid_composition(sequence)
        return composition


def aa_occurances(df):
    composition = [get_amino_acid_composition(sequence) for sequence in df['CDR3']]
    aa_alfa_counts = pd.DataFrame.from_records(composition).fillna(0)
    aa_alfa_counts.columns = [f'{column}_count' for column in aa_alfa_counts.columns]
    return aa_alfa_counts


feature_functions.append(aa_occurances)

aa_occurances(test_df).head()

,C_count,A_count,S_count,L_count,T_count,W_count,G_count,N_count,E_count,Q_count,F_count,Y_count,P_count,R_count,I_count,D_count,V_count,K_count,H_count,M_count
0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,2.0,2.0,1.0,2.0,0.0,2.0,1.0,2.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,3.0,0.0,1.0,0.0,1.0,0.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,2.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,3.0,2.0,0.0,1.0,0.0,1.0,1.0,2.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Average physicochemical properties

In [28]:
from util import PHYSCHEM_PROPERTIES


def get_property(sequence, prop_lookup):
    if not isinstance(sequence, str):
        # It's probably NaN
        return 0
    else:
        return np.mean(list(prop_lookup[aa] for aa in sequence))


def physchem_properties(df):
    physchem_df = pd.DataFrame()
    for prop_name, prop_lookup in PHYSCHEM_PROPERTIES.items():
        physchem_df[prop_name] = df['CDR3'].apply(get_property, args=(prop_lookup,))
    return physchem_df


feature_functions.append(physchem_properties)

physchem_properties(test_df).head()

,basicity,hydrophobicity,helicity,mutation_stability
0,210.340000,-0.165333,1.055333,21.333333
1,209.888235,-0.261765,1.074118,20.176471
2,213.637500,-1.021875,1.010625,18.250000
3,209.830769,1.012308,1.083077,24.384615
4,210.162500,0.021250,1.077500,19.750000


# peptide mass

In [29]:
from pyteomics import mass


def get_mass(sequence):
    if not isinstance(sequence, str):
        # It's probably NaN
        return 0
    else:
        return mass.fast_mass(sequence)

def peptide_mass(df):
    return df['CDR3'].apply(get_mass).to_frame('peptide_mass')

feature_functions.append(peptide_mass)

peptide_mass(test_df).head()

,peptide_mass
0,1689.709321
1,1823.767230
2,1828.852631
3,1445.690089
4,1770.719551


# pI

In [30]:
from pyteomics import electrochem


def get_pi(sequence):
    if not isinstance(sequence, str):
        return 0
    else:
        return electrochem.pI(sequence)


def pi_feature(df):
    return df['CDR3'].apply(get_pi).to_frame('pi')

feature_functions.append(pi_feature)

pi_feature(test_df).head()

,pi
0,3.298340
1,3.127441
2,8.842285
3,3.298340
4,3.127441


# Positional features (i.e. localized at  specific amino acid position)


In [31]:
from util import get_basicity, get_hydrophobicity, get_helicity, get_mutation_stability

def pos_features(df):
    features_list = []
    pos_aa, pos_basicity, pos_hydro, pos_helicity, pos_mutation, pos_pI = [[] for _ in range(6)]

    for sequence in df['CDR3']:
        if not isinstance(sequence, str):
            continue

        length = get_length(sequence)

        start_pos = -1 * (length // 2)

        # Ranges are averaged around 0, so if mod 2 = 1, we need to include 0, else not
        if length % 2 == 1:
            pos_range = list(range(start_pos, start_pos + length))
        else:
            pos_range = list(range(start_pos, 0)) + list(range(1, start_pos + length + 1))

        # bool 1 or 0 if amino acid is present at position
        pos_aa.append({f'pos_{pos}_{aa}': 1 for pos, aa in zip(pos_range, sequence)})

        pos_basicity.append({f'pos_{pos}_basicity': get_basicity(aa) for pos, aa in zip(pos_range, sequence)})
        pos_hydro.append({f'pos_{pos}_hydrophobicity': get_hydrophobicity(aa) for pos, aa in zip(pos_range, sequence)})
        pos_helicity.append({f'pos_{pos}_helicity': get_helicity(aa) for pos, aa in zip(pos_range, sequence)})
        pos_mutation.append({f'pos_{pos}_mutation_stability': get_mutation_stability(aa) for pos, aa in zip(pos_range, sequence)})

        pos_pI.append({f'pos_{pos}_pI': electrochem.pI(aa) for pos, aa in zip(pos_range, sequence)})

    features_list.append(pd.DataFrame.from_records(pos_aa).fillna(0))
    features_list.append(pd.DataFrame.from_records(pos_basicity).fillna(0))
    features_list.append(pd.DataFrame.from_records(pos_hydro).fillna(0))
    features_list.append(pd.DataFrame.from_records(pos_helicity).fillna(0))
    features_list.append(pd.DataFrame.from_records(pos_mutation).fillna(0))
    features_list.append(pd.DataFrame.from_records(pos_pI).fillna(0))

    return pd.concat(features_list, axis=1)

feature_functions.append(pos_features)

pos_features(test_df).head()

,pos_-7_C,pos_-6_A,pos_-5_S,pos_-4_S,pos_-3_L,pos_-2_T,pos_-1_S,pos_0_W,pos_1_G,pos_2_N,...,pos_13_pI,pos_14_pI,pos_15_pI,pos_16_pI,pos_-19_pI,pos_-18_pI,pos_-17_pI,pos_17_pI,pos_18_pI,pos_19_pI
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Combine all features

In [32]:
def get_sequence_features(df):
    features_list = []
    for feature_function in feature_functions:
        features_list.append(feature_function(df))
    return pd.concat(features_list, axis=1)

In [33]:
def get_features(df):
    beta_renamed = df[['CDR3_beta', 'TRBV', 'TRBJ']].rename(columns={'CDR3_beta': 'CDR3', 'TRBV': 'V', 'TRBJ': 'J'})
    beta_features = get_sequence_features(beta_renamed).add_prefix('beta_')

    alpha_renamed = df[['CDR3_alfa', 'TRAV', 'TRAJ']].rename(columns={'CDR3_alfa': 'CDR3', 'TRAV': 'V', 'TRAJ': 'J'})
    alpha_features = get_sequence_features(alpha_renamed).add_prefix('alfa_')

    return pd.concat([beta_features, alpha_features], axis=1)

In [34]:
features = get_features(df)
features.head()

,beta_J,beta_J=TRBJ1-1,beta_J=TRBJ1-1*01,beta_J=TRBJ1-2,beta_J=TRBJ1-2*01,beta_J=TRBJ1-3,beta_J=TRBJ1-3*01,beta_J=TRBJ1-4,beta_J=TRBJ1-4*01,beta_J=TRBJ1-5,...,alfa_pos_-9_pI,alfa_pos_-8_pI,alfa_pos_8_pI,alfa_pos_9_pI,alfa_pos_-10_pI,alfa_pos_10_pI,alfa_pos_-11_pI,alfa_pos_11_pI,alfa_pos_-12_pI,alfa_pos_12_pI
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
everything = pd.concat([df, features], axis=1)
everything.head()

,CDR3_alfa,TRAV,TRAJ,CDR3_beta,TRBV,TRBJ,reaction,beta_J,beta_J=TRBJ1-1,beta_J=TRBJ1-1*01,...,alfa_pos_-9_pI,alfa_pos_-8_pI,alfa_pos_8_pI,alfa_pos_9_pI,alfa_pos_-10_pI,alfa_pos_10_pI,alfa_pos_-11_pI,alfa_pos_11_pI,alfa_pos_-12_pI,alfa_pos_12_pI
0,NaN,NaN,NaN,CASSLTSWGNNEQFF,TRBV5-5,TRBJ2-1,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NaN,NaN,NaN,CASSLEATGGTYNEQFF,TRBV5-1,TRBJ2-1,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NaN,NaN,NaN,CASSSPRRGIQETQYF,TRBV5-1*01,TRBJ2-5*01,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,CAGPGGSSNTGKLIF,TRAV35*01,TRAJ37*01,CASSLIYPGELFF,TRBV27*01,TRBJ2-2*01,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NaN,NaN,NaN,CASSFEAGATYNEQFF,TRBV7-6,TRBJ2-1,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Summary
I implemented features based on the features in tcrexmodel.ipynb (under reference code). This consist of a one hot encoder (which is different when training (fit and transform) and testing  (transform only)), the sequence length, number of occurences of each amino acid, the average of some physicochemical properties, the peptide mass, pI, and some positional features.

Some of the features might create new (unseen) columns (e.g. the positional features), which is why we still need a fix_test function, which drops/create columns to match the columns of the training set.